
<h2><center><font color='black'>   Taking a Deep-Learning Dive with Keras </font></center></h2>

<img src='imgs/Deep_Dive_Keras_2.jpg' align='middle'/>

<img src='imgs/cnn1.png'/>

<img src='imgs/cnn_family_a.png'/>

<img src='imgs/cnn_family.png'/>
<img src='imgs/many_architectures.png'/>
* bubble size is # of parameters
* AlexNet  (62 million parameters) (final layer 6x 6 x 256 x 2096) ~ > 1/2 are coming from FC layer

<img src='imgs/input.png'/>

<img src='imgs/conv1.png'/>

<img src='imgs/conv_title.png'/>

<img src='imgs/3D_Convolution_Animation.gif'/>

**Definition of Convolution:**
Integral that expresses overlap of one function as it is shifted over another

**In terms of CNNs:**
The input is one function, and the kernel is the other. We can implement the integral as 
a summation over a finite # of array elements.  The output is called the feature map (which we can think of as a weighted sum)

<img src='imgs/conv_a.png'/>
<img src='imgs/conv_bb.png'/>

<img src='imgs/conv_gif_a.png'/>
<img src='imgs/conv_filt_b.png'/>


<img src='imgs/conv_layer_math.png'/>
<img src='imgs/conv_filter.png'/>

<img src='imgs/max_pool.png'/>


### Max-Pooling benefits: 

- Reduces parameters by 75% 
- Helps us to generalize our model: Provides an 'abstracted' form of our features

<img src='imgs/relu_1.png'/>

<img src='imgs/relu2.png'/>


<img src='imgs/fc.png'/>

<img src='imgs/all_layers.png'/>

Credits: [Yosinski](http://yosinski.com)




In [2]:
from keras import backend as K
import keras.utils.np_utils

from keras.models import Sequential
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D, ZeroPadding1D
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD
import numpy as np
import pandas as pd

from IPython.display import SVG
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import sys

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

##  And now,  CNN in Keras! 


In [3]:
import keras.backend as K
K.set_image_dim_ordering('th')
K.set_image_data_format('channels_first')

#### Macroarchitecture of VGG16
<img src='imgs/VGG.png'/>

In [4]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512,(3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [5]:
!curl https://raw.githubusercontent.com/torch/tutorials/master/7_imagenet_classification/synset_words.txt -o synset_words.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31675  100 31675    0     0   104k      0 --:--:-- --:--:-- --:--:--  104k


[Download weights](https://github.com/fchollet/deep-learning-models/releases/tag/v0.1)

In [6]:
model = VGG_16('~/Downloads/vgg16_weights_th_dim_ordering_th_kernels.h5')
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

In [7]:
synset = pd.read_csv('synset_words.txt', skipinitialspace=True, names = ['synset', 'words'])

In [8]:
def prepare_image(image_path):
    im = cv2.resize(cv2.imread(image_path), (224, 224)).astype(np.float32)
    # subtracting means across the color channels
    # (The mean pixel values are taken from the VGG authors)
    im[:,:,0] -= 103.939
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    im = im.transpose((2,0,1))
    im = np.expand_dims(im, axis=0)
    return im

<img src='imgs/dog2.jpg'/>

In [9]:
img = prepare_image('imgs/dog2.jpg')
out = model.predict(img)
y_pred = np.argmax(out)

print (y_pred)
print (synset.loc[y_pred].synset)

259
n02112018 Pomeranian


Close enough! :) 

<img src='imgs/sloth.jpg'/>

In [10]:
img = prepare_image('imgs/sloth.jpg')
out = model.predict(img)
y_pred = np.argmax(out)

print (y_pred)
print (synset.loc[y_pred].synset)

968
n07930864 cup


<img src='imgs/dress_two.png'/>

### Let's move over to our GPU for hands on practice with CNNs & Cifar:

- navigate to [rosettahub](https://www.rosettahub.com)    
(Make sure to use Chrome or Firefox browswer)

- click on Sign In (Use provided login & password)     

- Click on Formations (Upper Right Corner)    

Click 'yes' to launch the machine!




